In [2]:
import pandas as pd
from nltk.util import ngrams

import numpy as np
import os 
import unicodedata
#import transliterate
#from transliterate import translit, get_available_language_codes
import string
#import translit
#import romanize
import unicodedata

In [3]:
df  =  pd.read_excel('name_data/exigerData/EXGR_Japanese names.xlsx')
print(df)

        Unnamed: 0                                        id  \
0                0  http://www.wikidata.org/entity/Q17229484   
1                1  http://www.wikidata.org/entity/Q19958115   
2                2  http://www.wikidata.org/entity/Q11237697   
3                3  http://www.wikidata.org/entity/Q11482333   
4                4  http://www.wikidata.org/entity/Q11528757   
...            ...                                       ...   
202435      202435  http://www.wikidata.org/entity/Q11672225   
202436      202436    http://www.wikidata.org/entity/Q495816   
202437      202437  http://www.wikidata.org/entity/Q11640390   
202438      202438   http://www.wikidata.org/entity/Q3984842   
202439      202439  http://www.wikidata.org/entity/Q11362511   

                  fullname  Family name  Given name  
0       Hirotoshi Nakamura          NaN         NaN  
1          Sachio Hosokawa          NaN         NaN  
2                      OZA          NaN         NaN  
3              

In [4]:
#shape before doing anything (202440,5)
df.shape

(202440, 5)

In [5]:
df.head()

,Unnamed: 0,id,fullname,Family name,Given name
0,0,http://www.wikidata.org/entity/Q17229484,Hirotoshi Nakamura,NaN,NaN
1,1,http://www.wikidata.org/entity/Q19958115,Sachio Hosokawa,NaN,NaN
2,2,http://www.wikidata.org/entity/Q11237697,OZA,NaN,NaN
3,3,http://www.wikidata.org/entity/Q11482333,平塚利男,NaN,NaN
4,4,http://www.wikidata.org/entity/Q11528757,Jun Kochi,NaN,NaN


In [6]:
#cheaking how many null value each columns have 
nan_count = np.sum(df.isnull(), axis = 0)
nan_count


Unnamed: 0          0
id                  0
fullname            1
Family name    202440
Given name     202440
dtype: int64

In [7]:
#checking if full name have any duplicates 
df['fullname'].duplicated().any()

True

In [8]:
#create a new variable and drop the duplicates
df2 =df.drop_duplicates(subset=['fullname'])
print(df2)

        Unnamed: 0                                        id  \
0                0  http://www.wikidata.org/entity/Q17229484   
1                1  http://www.wikidata.org/entity/Q19958115   
2                2  http://www.wikidata.org/entity/Q11237697   
3                3  http://www.wikidata.org/entity/Q11482333   
4                4  http://www.wikidata.org/entity/Q11528757   
...            ...                                       ...   
202435      202435  http://www.wikidata.org/entity/Q11672225   
202436      202436    http://www.wikidata.org/entity/Q495816   
202437      202437  http://www.wikidata.org/entity/Q11640390   
202438      202438   http://www.wikidata.org/entity/Q3984842   
202439      202439  http://www.wikidata.org/entity/Q11362511   

                  fullname  Family name  Given name  
0       Hirotoshi Nakamura          NaN         NaN  
1          Sachio Hosokawa          NaN         NaN  
2                      OZA          NaN         NaN  
3              

In [9]:
# making sure all duplicates are gone 
df2['fullname'].duplicated().any()

False

In [10]:
#checking if the shape went down(went form (202440,5) to (193534,5))
df2.shape

(193534, 5)

In [11]:
# checking if the null values went down (it did) 
nan_count = np.sum(df2.isnull(), axis = 0)
nan_count

Unnamed: 0          0
id                  0
fullname            1
Family name    193534
Given name     193534
dtype: int64

In [12]:
#printing the col_names with null values
condition = nan_count != 0
col_names = nan_count[condition].index
nan_cols = list(col_names)
print(nan_cols)

print(df2['id'].unique())
print(df2['fullname'].unique())
print(df2['Family name'].unique())
print(df2['Given name'].unique())

['fullname', 'Family name', 'Given name']
['http://www.wikidata.org/entity/Q17229484'
 'http://www.wikidata.org/entity/Q19958115'
 'http://www.wikidata.org/entity/Q11237697' ...
 'http://www.wikidata.org/entity/Q11460373'
 'http://www.wikidata.org/entity/Q11384166'
 'http://www.wikidata.org/entity/Q11384100']
['Hirotoshi Nakamura' 'Sachio Hosokawa' 'OZA' ... 'Ryō Yuzawa' '菅原照仁'
 '世羅りさ']
[nan]
[nan]


In [14]:
#create new variable that has the dataframe with the drop column
df2 = df2.drop(columns = ['Unnamed: 0'])

nan_count = np.sum(df2.isnull(), axis = 0)
nan_count


KeyError: "['Unnamed: 0'] not found in axis"

In [17]:
#fullname still has one nan value 
df2[df2['fullname'].isna()]

,id,fullname,Family name,Given name


In [16]:
#dropping that nan value for good 
df2.dropna(subset=['fullname'], inplace= True)

nan_count = np.sum(df2.isnull(), axis = 0)
nan_count

id                  0
fullname            0
Family name    193533
Given name     193533
dtype: int64

In [18]:
df2[df2['fullname'].isna()]

,id,fullname,Family name,Given name


In [38]:
non_alnum_names_japanese = [name for name in df2['fullname'] if isinstance(name, str) and not name.replace(' ', '').isalnum()]
print(len(non_alnum_names_japanese))
non_alnum_names_japanese


4806


['佐藤直樹 (日活)',
 '島田陽子 (詩人)',
 'Fushimi-no-miya Sadakiyo-shinnō',
 '加藤歩 (アナウンサー)',
 '菅原健 (イラストレーター)',
 "Yūza San'yūtei",
 "Shin'ichi Tamura",
 'AKIRA (プロレスラー)',
 '朝日稔 (動物学者)',
 "Gen'ichirō Kakegawa",
 "Gen'e Imai",
 '山田泉 (教育者)',
 'Shigeakira-shinnō',
 'bird (日本の歌手)',
 '優希 (声優)',
 '笑福亭松鶴 (6代目)',
 '斎藤修 (一橋大学)',
 '松本龍 (アナウンサー)',
 'Tokiwai-no-miya Tsunenao-shinnō',
 "Kan'ichi Kawakami",
 "Shin'ichirō Ishiwatari",
 '江藤智 (政治家)',
 'T-岡田',
 "Kin'ichi Aoki",
 '阿修羅・原',
 '桃子 (AV女優)',
 'Enshi-naishinnō',
 "Ken'ichiro Kuroiwa",
 '清水哲也 (医学者)',
 "Shin'ichirō Hara",
 'ジェイムズ・アベグレン',
 "Ken'ichi Tsukamoto",
 "Ken'ichirō Hamada",
 'Rihō-joō',
 "Shin'ya Ōwada",
 '三浦大輔 (雀士)',
 "Shōko Nihon'yanagi",
 '井上薫 (弁護士)',
 'み〜こ',
 '伊藤武彦 (内務官僚)',
 'Nishimura Tei (government official)',
 'トゥイ・ティエン',
 '河野太郎 (美術家)',
 "Shin'ichi Kimura",
 "Gen'ichirō Ōshita",
 '2C = Galore',
 '佐藤まさみ (声優)',
 'シギ (歌手)',
 "Ken'ichirō Masaoka",
 "Jun'ichi Akimoto",
 '坂本茂 (野球)',
 "Kan'ichirō Shinobu",
 '中村俊介 (新聞記者)',
 '前田和之 (サッカー選手)',
 'Ooshima,

In [41]:
#checking the char_ngrams 
def japanese_char_ngrams(names,n):
    #checks for 
    if not isinstance(names,str):
        return []
    
    char_ngrams = ngrams(names, n)
    return [''.join(gram) for gram in char_ngrams]

n = 3
df2['char_ngrams'] = df2['fullname'].apply(lambda x: japanese_char_ngrams(x, n))

In [20]:
# feature to check the name length
def name_length(name):
    if isinstance(name,str):
        return len(name.replace(' ', ''))
    else:
        return None


df2['name_len'] = df2['fullname'].apply(name_length)
print(df2['name_len'])

0         17.0
1         14.0
2          3.0
3          4.0
4          8.0
          ... 
202435    12.0
202436    15.0
202437     9.0
202438     4.0
202439     4.0
Name: name_len, Length: 193533, dtype: float64


In [19]:
# feature to check the token length
def token_length(name):
    if isinstance(name,str):
       return len(name.split())
    else:
       return None

df2['token_len'] = df2['fullname'].apply(token_length)
print(df2['token_len'])

0         2.0
1         2.0
2         1.0
3         1.0
4         2.0
         ... 
202435    2.0
202436    2.0
202437    2.0
202438    1.0
202439    1.0
Name: token_len, Length: 193533, dtype: float64


In [21]:
def japanese_lan(name):
    if isinstance(name, str):
        return [unicodedata.name(char).split(' ')[0] for char in name]
    else:
        return None

# Apply the function only to rows where 'fullname' is a string
df2['determine_alphabet'] = df2['fullname'].apply(japanese_lan)
print(df2['determine_alphabet'])


0         [LATIN, LATIN, LATIN, LATIN, LATIN, LATIN, LAT...
1         [LATIN, LATIN, LATIN, LATIN, LATIN, LATIN, SPA...
2                                     [LATIN, LATIN, LATIN]
3                                      [CJK, CJK, CJK, CJK]
4         [LATIN, LATIN, LATIN, SPACE, LATIN, LATIN, LAT...
                                ...                        
202435    [LATIN, LATIN, LATIN, LATIN, LATIN, LATIN, SPA...
202436    [LATIN, LATIN, LATIN, LATIN, LATIN, LATIN, LAT...
202437    [LATIN, LATIN, LATIN, SPACE, LATIN, LATIN, LAT...
202438                                 [CJK, CJK, CJK, CJK]
202439                       [CJK, CJK, HIRAGANA, HIRAGANA]
Name: determine_alphabet, Length: 193533, dtype: object


In [66]:
df2.shape
df2.head()

,id,fullname,Family name,Given name,char_ngrams,name_len,token_len,determine_alphabet
0,http://www.wikidata.org/entity/Q17229484,Hirotoshi Nakamura,NaN,NaN,"[Hir, iro, rot, oto, tos, osh, shi, hi , i N, ...",17.0,2.0,"[LATIN, LATIN, LATIN, LATIN, LATIN, LATIN, LAT..."
1,http://www.wikidata.org/entity/Q19958115,Sachio Hosokawa,NaN,NaN,"[Sac, ach, chi, hio, io , o H, Ho, Hos, oso, ...",14.0,2.0,"[LATIN, LATIN, LATIN, LATIN, LATIN, LATIN, SPA..."
2,http://www.wikidata.org/entity/Q11237697,OZA,NaN,NaN,[OZA],3.0,1.0,"[LATIN, LATIN, LATIN]"
3,http://www.wikidata.org/entity/Q11482333,平塚利男,NaN,NaN,"[平塚利, 塚利男]",4.0,1.0,"[CJK, CJK, CJK, CJK]"
4,http://www.wikidata.org/entity/Q11528757,Jun Kochi,NaN,NaN,"[Jun, un , n K, Ko, Koc, och, chi]",8.0,2.0,"[LATIN, LATIN, LATIN, SPACE, LATIN, LATIN, LAT..."


In [22]:
text = "Lorem ipsum dolor sit amet"
print(translit(text, 'hy'))

NameError: name 'translit' is not defined